# Starter Tutorial
This tutorial provides example code for getting started using **EDSL**, an open-source Python library for simulating surveys, experiments and other research tasks with AI agents and large language models. EDSL is available under the MIT License at [PyPI](https://pypi.org/project/edsl/) and [GitHub](https://github.com/expectedparrot/edsl). 

In the steps below we show how to construct and run a simple question in EDSL, and then how to design more complex surveys with AI agent personas and multiple language models.
We also demonstrate methods for applying logic and rules to surveys, piping answers into questions, adding data to questions and analyzing survey results as datasets.

We also show how to post and share content that you create at **Coop**, a platform for creating, storing and collaborating on AI-based research. Coop is fully integrated with EDSL and free to use. Create an account [here](https://www.expectedparrot.com/login).

Please also see our [documentation page](https://docs.expectedparrot.com) for more details on the topics covered in this notebook.
If you encounter any issues or have questions, please email us at info@expectedparrot.com or post a question at our [Discord channel](https://discord.com/invite/mxAYkjfy9m).

## Technical setup
To run the examples below you first need to install the EDSL library and choose how you want to access language models.

*Note: If you are using EDSL in Google Colab, please see the [Colab setup](https://docs.expectedparrot.com/en/latest/colab_setup.html) page for additional instructions.*


### Install EDSL
Uncomment and run the following code to install the EDSL library. 
See installation [instructions](https://docs.expectedparrot.com/en/latest/installation.html) for more details.

In [1]:
# pip install edsl

If you have already installed EDSL, you can uncomment and run the following code to check that your version is up to date (compare it to the version at [PyPI](https://pypi.org/project/edsl/)):

In [2]:
# pip show edsl

If your version of EDSL is not up to date, uncomment and run the following code to update it:

In [3]:
# pip install --upgrade edsl

### Choose how to access language models
Next, decide how you want to access language models with EDSL and obtain the required API keys:

* *Remote inference:* This method allows you to run surveys at the Expected Parrot server and access all available language models at once with your Expected Parrot API key. [Learn more](https://docs.expectedparrot.com/en/latest/remote_inference.html).
  
* *Local inference:* Alternatively, you can obtain your own API keys for service providers to run EDSL on your own machine. 

Your Expected Parrot API key can be found at the Settings page of your [Coop account](https://www.expectedparrot.com/login), where you can select the option to [activate remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html).
This key also allows you to [post and share content at the Coop](https://www.expectedparrot.com/content/explore) that you create using either remote or local inference. 

### Store your API keys
Make your keys available to EDSL by storing then in a file named "**.env**" in your working directory. 

***Note: Your API keys should be treated like passwords and deleted from notebooks or content that you share with others. We recommend deleting your keys from the code below after your .env file has been created.***

## Example: Running a simple question
EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model.
To see a list of all question types:

In [1]:
from edsl import Question

Question.available()

,question_type,question_class,example_question
0,checkbox,QuestionCheckBox,"Question('checkbox', question_name = """"""never_eat"""""", question_text = """"""Which of the following foods would you eat if you had to?"""""", min_selections = 2, max_selections = 5, question_options = ['soggy meatpie', 'rare snails', 'mouldy bread', 'panda milk custard', 'McDonalds'], include_comment = False, use_code = True)"
1,extract,QuestionExtract,"Question('extract', question_name = """"""extract_name"""""", question_text = """"""My name is Moby Dick. I have a PhD in astrology, but I'm actually a truck driver"""""", answer_template = {'name': 'John Doe', 'profession': 'Carpenter'})"
2,free_text,QuestionFreeText,"Question('free_text', question_name = """"""how_are_you"""""", question_text = """"""How are you?"""""")"
3,functional,QuestionFunctional,"Question('functional', question_name = """"""sum_and_multiply"""""", question_text = """"""Calculate the sum of the list and multiply it by the agent trait multiplier."""""")"
4,likert_five,QuestionLikertFive,"Question('likert_five', question_name = """"""happy_raining"""""", question_text = """"""I'm only happy when it rains."""""", question_options = ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree'])"
5,linear_scale,QuestionLinearScale,"Question('linear_scale', question_name = """"""ice_cream"""""", question_text = """"""How much do you like ice cream?"""""", question_options = [1, 2, 3, 4, 5], option_labels = {1: 'I hate it', 5: 'I love it'})"
6,list,QuestionList,"Question('list', question_name = """"""list_of_foods"""""", question_text = """"""What are your favorite foods?"""""", max_list_items = None)"
7,matrix,QuestionMatrix,"Question('matrix', question_name = """"""child_happiness"""""", question_text = """"""How happy would you be with different numbers of children?"""""", question_items = ['No children', '1 child', '2 children', '3 or more children'], question_options = [1, 2, 3, 4, 5], option_labels = {1: 'Very sad', 3: 'Neutral', 5: 'Extremely happy'})"
8,multiple_choice,QuestionMultipleChoice,"Question('multiple_choice', question_name = """"""how_feeling"""""", question_text = """"""How are you?"""""", question_options = ['Good', 'Great', 'OK', 'Bad'], include_comment = False)"
9,numerical,QuestionNumerical,"Question('numerical', question_name = """"""age"""""", question_text = """"""You are a 45 year old man. How old are you in years?"""""", min_value = 0, max_value = 86.7, include_comment = False)"


We can see the components of a particular question type by importing the question type class and calling the `example` method on it:

In [2]:
from edsl import (
    # QuestionCheckBox,
    # QuestionExtract,
    # QuestionFreeText,
    # QuestionFunctional,
    # QuestionLikertFive,
    # QuestionLinearScale,
    # QuestionList,
    QuestionMultipleChoice,
    # QuestionNumerical,
    # QuestionRank,
    # QuestionTopK,
    # QuestionYesNo
)

q = QuestionMultipleChoice.example() # substitute any question type class name
q

,key,value
0,question_name,how_feeling
1,question_text,How are you?
2,question_options:0,Good
3,question_options:1,Great
4,question_options:2,OK
5,question_options:3,Bad
6,include_comment,False
7,question_type,multiple_choice


Here we create a simple multiple choice question of our own:

In [3]:
from edsl import QuestionMultipleChoice

q = QuestionMultipleChoice(
    question_name = "smallest_prime",
    question_text = "Which is the smallest prime number?",
    question_options = [0, 1, 2, 3]
)

We can administer the question to a language model by calling the `run` method on it. 
If you have activated remote inference and stored your Expected Parrot API key (see instructions above), the question will be run remotely at the Expected Parrot server.
Results are stored at an unlisted Coop page by default; we can also set the visibility to `public` or `private` either when we run it or by updating the object (demonstrated in later examples).
We can also view a progress report for the job:

In [4]:
results = q.run()

/Users/johnhorton/tools/edsl/edsl/coop/ExpectedParrotKeyHandler.py:92: UserWarning: WARNING: The Expected Parrot API key from the environment variable differs from the one stored in the config directory. Using the one from the environment variable.
  warnings.warn(


Job UUID,2df5e752-2ec5-46d8-9504-7889c2aab69c
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/2df5e752-2ec5-46d8-9504-7889c2aab69c
Error Report URL,None
Results UUID,c7714eca-a7cd-4478-945f-e4feb94ed2fa
Results URL,None


Updating local cache with 21 new entries from remote...
Local cache updated!
No new entries to add to remote cache.
There are 10,145 entries in the local cache.


### Inspecting results
This generates a dataset of `Results` that we can readily access with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html). 
Here we `select()` the response to inspect it, together with the model that was used and the model's "comment" about its response--a field that is automatically added to all question types other than free text:

In [9]:
results.select("model", "smallest_prime", "smallest_prime_comment")

,model.model,answer.smallest_prime,comment.smallest_prime_comment
0,gpt-4o,2,2 is the smallest prime number because it is greater than 1 and has no divisors other than 1 and itself.


The `Results` also include information about the question, model parameters, prompts, generated tokens and raw responses. 
To see a list of all the components:

In [10]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.smallest_prime
4,cache_used.smallest_prime_cache_used
5,comment.smallest_prime_comment
6,generated_tokens.smallest_prime_generated_tokens
7,iteration.iteration
8,model.frequency_penalty
9,model.logprobs


## Example: Conducting a survey with agents and models
In the next example we construct a more complex survey consisting of multiple questions, and design personas for AI agents to answer the survey.
Then we select specific language models to generate the answers.

We start by creating questions in different types and passing them to a `Survey`:

In [30]:
from edsl import QuestionLinearScale, QuestionFreeText

q_enjoy = QuestionLinearScale(
    question_name = "enjoy",
    question_text = "On a scale from 1 to 5, how much do you enjoy reading?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place",
    question_text = "Describe your favorite place for reading."
)

We construct a `Survey` by passing a list of questions:

In [31]:
from edsl import Survey

survey = Survey(questions = [q_enjoy, q_favorite_place])

### Agents
An important feature of EDSL is the ability to create AI agents to answer questions.
This is done by passing dictionaries of relevant "traits" to `Agent` objects that are used by language models to generate responses.
Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html).

Here we construct several simple agent personas to use with our survey:

In [32]:
from edsl import AgentList, Agent

agents = AgentList(
    Agent(traits = {"persona":p}) for p in ["artist", "mechanic", "sailor"]
)

### Language models
EDSL works with many popular large language models that we can select to use with a survey.
This makes it easy to compare responses among models in the results that are generated.

To see a current list of available models:

In [33]:
from edsl import Model

Model.available() 

/Users/johnhorton/tools/edsl/edsl/inference_services/AvailableModelFetcher.py:139: UserWarning: No models found for service ollama
  warnings.warn(f"No models found for service {service_name}")


,Model Name,Service Name
0,Austism/chronos-hermes-13b-v2,deep_infra
1,Gryphe/MythoMax-L2-13b,deep_infra
2,Qwen/Qwen2-72B-Instruct,deep_infra
3,Qwen/Qwen2-7B-Instruct,deep_infra
4,Qwen/Qwen2.5-72B-Instruct,deep_infra
5,Sao10K/L3-70B-Euryale-v2.1,deep_infra
6,Sao10K/L3.1-70B-Euryale-v2.2,deep_infra
7,google/gemma-2-27b-it,deep_infra
8,google/gemma-2-9b-it,deep_infra
9,lizpreciatior/lzlv_70b_fp16_hf,deep_infra


To check the default model that will be used if no models are specified for a survey (e.g., as in the first example above):

In [35]:

Model.services()

,Service Name,Local key?
0,openai,yes
1,anthropic,yes
2,deep_infra,yes
3,google,yes
4,groq,yes
5,bedrock,
6,azure,
7,ollama,
8,test,yes
9,together,


(Note that the output may be different if the default model has changed since this page was last updated.)

Here we select some models to use with our survey:

In [28]:
from edsl import ModelList, Model

models = ModelList(
    Model(m) for m in ["gpt-4o", "gemini-1.5-flash"]
)

### Running a survey
We add agents and models to a survey using the `by` method.
Then we administer a survey the same way that we do an individual question, by calling the `run` method on it:

In [29]:
results = survey.by(agents).by(models).run()

/Users/johnhorton/tools/edsl/edsl/agents/QuestionInstructionPromptBuilder.py:94: UserWarning: Question instructions still has variables: ['activity'].
  warn(f"Question instructions still has variables: {undefined_vars}.")


Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect A

KeyboardInterrupt: 

Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect A

Exceptions were raised in 6 interviews.
Exceptions were raised in the following interviews: [0, 1, 2, 3, 4, 5].



Exception report saved to /Users/johnhorton/tools/edsl/docs/notebooks/tmp62xncgd2.html
Also see: https://docs.expectedparrot.com/en/latest/exceptions.html


We can pass an expression to `filter()` the results and list the components to `sort_by()`:

In [18]:
(
    results
    .filter("persona != 'artist'")
    .sort_by("persona", "model")
    .select("model", "persona", "enjoy", "favorite_place")
)

,model.model,agent.persona,answer.enjoy,answer.favorite_place
0,gemini-1.5-flash,mechanic,nan,nan
1,gpt-4o,mechanic,nan,nan
2,gemini-1.5-flash,sailor,nan,nan
3,gpt-4o,sailor,nan,nan


## Example: Adding context to questions
EDSL provides a variety of ways to add data or content to survey questions. 
These methods include:

* [Piping](https://docs.expectedparrot.com/en/latest/surveys.html#id2) answers to questions into follow-on questions
* [Adding "memory"](https://docs.expectedparrot.com/en/latest/surveys.html#question-memory) of prior questions and answers in a survey when presenting other questions to a model
* [Parameterizing questions with data](https://docs.expectedparrot.com/en/latest/scenarios.html), e.g., content from PDFs, CSVs, docs, images or other sources that you want to add to questions

### Piping question answers
Here we demonstrate how to pipe the answer to a question into the text of another question.
This is done by using a placeholder `{{ <question_name>.answer }}` in the text of the follow-on question where the answer to the prior question is to be inserted when the survey is run.
This causes the questions to be administered in the required order (survey questions are administered asynchronously by default).
Learn more about [piping question answers](https://docs.expectedparrot.com/en/latest/surveys.html#id2).

Here we insert the answer to a numerical question into the text of a follow-on yes/no question:

In [19]:
from edsl import QuestionNumerical, QuestionYesNo, Survey

q1 = QuestionNumerical(
    question_name = "random_number",
    question_text = "Pick a random number between 1 and 1,000."
)

q2 = QuestionYesNo(
    question_name = "prime",
    question_text = "Is this a prime number: {{ random_number.answer }}"
)

survey = Survey([q1, q2])

results = survey.run()

We can check the `user_prompt` for the `prime` question to verify that that the answer to the `random_number` question was piped into it:

In [20]:
results.select("random_number", "prime_user_prompt", "prime", "prime_comment")

,answer.random_number,prompt.prime_user_prompt,answer.prime,comment.prime_comment
0,487,"Is this a prime number: 487 No Yes Only 1 option may be selected. Please respond with just your answer. After the answer, you can put a comment explaining your response.",No,"487 is not a prime number because it can be divided by 1, 487, and also by 19 and 25."


### Adding "memory" of questions and answers
Here we instead add a "memory" of the first question and answer to the context of the second question.
This is done by calling a memory rule and identifying the question(s) to add.
Instead of just the answer, information about the full question and answer are presented with the follow-on question text, and no placeholder is used.
Learn more about [question memory rules](https://docs.expectedparrot.com/en/latest/surveys.html#survey-rules-logic).

Here we demonstrate the `add_targeted_memory` method (we could also use `set_full_memory_mode` or other memory rules):

In [21]:
from edsl import QuestionNumerical, QuestionYesNo, Survey

q1 = QuestionNumerical(
    question_name = "random_number",
    question_text = "Pick a random number between 1 and 1,000."
)

q2 = QuestionYesNo(
    question_name = "prime",
    question_text = "Is the number you picked a prime number?"
)

survey = Survey([q1, q2]).add_targeted_memory(q2, q1)

results = survey.run()

Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect A

Exceptions were raised in 1 interviews.



Also see: https://docs.expectedparrot.com/en/latest/exceptions.html


We can again use the `user_prompt` to verify the context that was added to the follow-on question. To view the results in a long table, we can call the `table()` and `long()` methods to modify the default table view:

In [22]:
results.select("random_number", "prime_user_prompt", "prime", "prime_comment").table().long()

,row,key,value
0,0,answer.random_number,487
1,0,prompt.prime_user_prompt,"Is the number you picked a prime number? No Yes Only 1 option may be selected. Please respond with just your answer. After the answer, you can put a comment explaining your response. Before the question you are now answering, you already answered the following question(s): Question: Pick a random number between 1 and 1,000. Answer: None"
2,0,answer.prime,None
3,0,comment.prime_comment,Task failed with exception: Language model did not return a response for question 'prime.'.


*Related topic: Learn more about exploring and simulating "randomness" with AI agents and LLMs in [this notebook](https://docs.expectedparrot.com/en/latest/notebooks/random_numbers.html).*

## Scenarios
We can also add external data or content to survey questions.
This can be useful when you want to efficiently create and administer multiple versions of questions at once, e.g., for conducting data labeling tasks.
This is done by creating `Scenario` dictionaries for the data or content to be used with a survey, where the keys match `{{ placeholder }}` names used in question texts (or question options) and the values are the content to be added.
Scenarios can also be used to [add metadata to survey results](https://docs.expectedparrot.com/en/latest/notebooks/adding_metadata.html), e.g., data sources or other information that you may want to include in the results for reference but not necessarily include in question texts.

In the next example we revise the prior survey questions about reading to take a parameter for other activities that we may want to add to the questions, and create simple scenarios for some activities.
EDSL provides methods for automatically generating scenarios from a variety of data sources, including PDFs, CSVs, docs, images, tables and dicts. 
We use the `from_list` method to convert a list of activities into scenarios.

Then we demonstrate how to use scenarios to create multiple versions of our questions either (i) when constructing a survey or (ii) when running it:

* In the latter case, the `by` method is used to add scenarios to a survey of questions with placeholders at the time that it is run (the same way that agents and models are added to a survey). This adds a `scenario` column to the results with a row for each answer to each question for each scenario.
* In the former case, the `loop` method is used to create a list of versions of a question with the scenarios already added to it; when the questions are passed to a survey and it is run, the results include columns for each individual question; there is no `scenario` column and a single row for each agent's answers to all the questions.

Learn more about [using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

Here we create scenarios for a simple list of activities:

In [23]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList.from_list("activity", ["reading", "running", "relaxing"])

### Adding scenarios using the `by` method
Here we add the scenarios to the survey when we run it, together with any desired agents and models:

In [24]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey

q_enjoy = QuestionLinearScale(
    question_name = "enjoy",
    question_text = "On a scale from 1 to 5, how much do you enjoy {{ activity }}?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place",
    question_text = "In a brief sentence, describe your favorite place for {{ activity }}."
)

survey = Survey([q_enjoy, q_favorite_place])

In [25]:
results = survey.by(scenarios).by(agents).by(models).run()

Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


KeyboardInterrupt: 

Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_key*here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error code: 401 - {'error': {'message': 'Incorrect A

Exceptions were raised in 18 interviews.
Exceptions were raised in the following interviews: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17].



Exception report saved to /Users/johnhorton/tools/edsl/docs/notebooks/tmptgj65x6_.html
Also see: https://docs.expectedparrot.com/en/latest/exceptions.html


In [ ]:
(
    results
    .filter("model.model == 'gpt-4o'")
    .sort_by("activity", "persona")
    .select("activity", "persona", "enjoy", "favorite_place")
)

### Adding scenarios using the `loop` method
Here we add scenarios to questions when constructing a survey, as opposed to when running it.
When we run the survey the results will include columns for each question and no `scenario` field. 
Note that we can also optionally use the scenario key in the question names (they are otherwise incremented by default):

In [ ]:
from edsl import QuestionLinearScale, QuestionFreeText

q_enjoy = QuestionLinearScale(
    question_name = "enjoy_{{ activity }}", # optional use of scenario key
    question_text = "On a scale from 1 to 5, how much do you enjoy {{ activity }}?",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q_favorite_place = QuestionFreeText(
    question_name = "favorite_place_{{ activity }}", # optional use of scenario key
    question_text = "In a brief sentence, describe your favorite place for {{ activity }}."
)

Looping the scenarios to create lists of questions:

In [ ]:
enjoy_questions = q_enjoy.loop(scenarios)
enjoy_questions

In [ ]:
favorite_place_questions = q_favorite_place.loop(scenarios)
favorite_place_questions

Combining the questions in a survey:

In [ ]:
survey = Survey(questions = enjoy_questions + favorite_place_questions)

In [ ]:
results = survey.by(agents).by(models).run()

We can see that there are additional question fields and no "scenario" field:

In [ ]:
results.columns 

In [ ]:
(
    results
    .filter("model.model == 'gpt-4o'")
    .sort_by("persona")
    .select("persona", "enjoy_reading", "enjoy_running", "enjoy_relaxing", "favorite_place_reading", "favorite_place_running", "favorite_place_relaxing")
)

## Exploring `Results`
EDSL comes with [built-in methods for analyzing and visualizing survey results](https://docs.expectedparrot.com/en/latest/language_models.html). 
For example, you can call the `to_pandas` method to convert results into a dataframe:

In [ ]:
df = results.to_pandas(remove_prefix=True)
df

The `Results` object also supports SQL-like queries with the the `sql` method:

In [ ]:
results.sql("""
select model, persona, enjoy_reading, favorite_place_reading
from self
order by 1,2,3
""")

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

We can post any EDSL object to the Coop by call the `push` method on it, optionally passing a `description` and `visibility` status:

In [ ]:
results.push(description = "Starter tutorial sample survey results", visibility="public")

We can also post this notebook:

In [ ]:
from edsl import Notebook

notebook = Notebook(path="starter_tutorial.ipynb")

info = notebook.push(description="Starter Tutorial", visibility="public")
info

To update an object:

In [ ]:
notebook = Notebook(path="starter_tutorial.ipynb") # resave

notebook.patch(uuid = info["uuid"], value = notebook)